In [33]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from unet import UNet  # 假设 UNet 是 unet.py 中定义的一个类
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os
from __future__ import print_function, division
import os
from PIL import Image
import torch
import torch.utils.data
import torchvision.transforms as transforms
from torchvision.transforms import functional as F
from skimage import io
from torch.utils.data import Dataset

import numpy as np
import dataset



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_transform = transforms.Compose([
    transforms.ToPILImage(),  # 如果图像不是PIL图像则需要转换
    transforms.Resize((256, 1024)),  # 调整图像大小
    transforms.ToTensor(),  # 将图像转换为torch.Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

# 掩膜的转换操作，掩膜不需要标准化
mask_transform = transforms.Compose([
    transforms.ToPILImage(),  # 如果掩膜不是PIL图像则需要转换
    transforms.Resize((256, 1024), interpolation=transforms.InterpolationMode.NEAREST),  # 调整大小，使用最近邻插值
    transforms.ToTensor()  # 将掩膜转换为torch.Tensor
])

In [38]:

trainset = dataset.KITTIDataset('tinySet/img', 'tinySet/mask', transform=image_transform, mask_transform=mask_transform,device=device)
dataloader = DataLoader(trainset, batch_size=4, shuffle=True)

model = UNet(in_ch=3,out_ch=22).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



In [41]:
for i, (images, masks) in enumerate(dataloader):
     print(masks.type())
     print(images.shape)
     break

torch.FloatTensor
torch.Size([4, 3, 256, 1024])


In [42]:
def train(model, loader, optimizer, criterion, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        for i, (images, masks) in enumerate(loader):

            masks = masks.long()
            optimizer.zero_grad()
            outputs = model(images)
            masks = masks.squeeze(1) 
            loss = criterion(outputs, masks)  
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
train(model, dataloader, optimizer, criterion, 10)
# 保存整个模型
torch.save(model, 'model_complete.pth')

Epoch 1, Loss: 3.2100603580474854
Epoch 2, Loss: 3.0265517234802246
Epoch 3, Loss: 2.8276712894439697
Epoch 4, Loss: 2.6669654846191406
Epoch 5, Loss: 2.5580217838287354
Epoch 6, Loss: 2.5067832469940186
Epoch 7, Loss: 2.4736194610595703
Epoch 8, Loss: 2.4299087524414062
Epoch 9, Loss: 2.3780429363250732
Epoch 10, Loss: 2.3519949913024902


In [51]:
import os
import torch
from torchvision.utils import save_image
import numpy as np

def predict_and_save(model, loader, save_dir):
    model.eval()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    with torch.no_grad():
        for images, filenames in loader:
            # 假设你的模型和数据都在同一设备上
            outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)
            for i, filename in enumerate(filenames):
                # 将每张预测的掩膜转换为 numpy 数组
                mask = predicted_masks[i].cpu().numpy().astype(np.uint8)
                # mask+=10
                print(mask)

                # 将 mask 转换为 PIL 图像并保存
                pil_image = Image.fromarray(mask)
                save_path = os.path.join(save_dir, os.path.splitext(filename)[0] + '.png')
                pil_image.save(save_path)

# 初始化和调用你的模型和数据加载器
test_dataset = dataset.TestDataset('tinySet/img', transform=image_transform,device=device)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
save_masks_dir = 'predicted_masks'
predict_and_save(model, test_loader, save_masks_dir)


[[13 13 13 ... 13 13 13]
 [10 13 13 ... 10 14 13]
 [10 10 10 ... 10 10 13]
 ...
 [10 10 10 ... 10 10 13]
 [10 10 10 ... 10 10 14]
 [10 10 10 ... 10 10 18]]
[[13 13 13 ... 13 13 13]
 [13 13 13 ... 13 13 13]
 [10 10 10 ... 10 18 13]
 ...
 [10 10 10 ... 10 25 13]
 [10 10 10 ... 10 13 18]
 [10 10 10 ... 28 13 13]]
[[18 10 10 ... 10 10 13]
 [10 29 10 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 ...
 [10 10 10 ... 10 13 13]
 [10 10 10 ... 10 13 13]
 [10 10 10 ... 10 13 13]]
[[13 13 29 ... 13 10 10]
 [10 10 10 ... 10 10 10]
 [10 10 10 ... 13 10 13]
 ...
 [10 10 10 ... 10 25 13]
 [10 10 10 ... 10 13 18]
 [10 10 10 ... 28 13 13]]
[[13 13 10 ... 13 13 13]
 [13 13 13 ... 13 13 13]
 [10 10 10 ... 10 18 13]
 ...
 [10 10 10 ... 10 25 13]
 [10 13 18 ... 10 13 13]
 [10 10 10 ... 28 13 13]]
[[13 13 13 ... 13 13 13]
 [28 10 10 ... 10 22 13]
 [28 10 10 ... 25 13 13]
 ...
 [10 10 25 ... 10 10 10]
 [10 10 10 ... 10 10 10]
 [10 10 10 ... 28 10 10]]
[[18 10 10 ... 10 13 13]
 [10 29 10 ... 13 13 13]
 [10 10 10 ...